In [4]:
#library imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
import warnings
from matplotlib.pyplot import figure
warnings.filterwarnings('ignore')
from sklearn.linear_model import LinearRegression

In [8]:
#data imports
returns = pd.read_pickle("../Data/returns.pkl")
volumes = pd.read_csv('../Data/stock_volumes.csv', index_col = 'date', parse_dates = True)
info = pd.read_csv('../Data/stock_info.csv', index_col = 'Instrument')
sp_listings = pd.read_csv('../Data/sp500_listings.csv', index_col = 'date', parse_dates = True)

### Pipeline psuedo code
    for t in range (first_date to last_date):
        find stocks in investable universe at t (stocks in the S&P500 that have prices  
                                                 recorded for the last n days)
    
        for stock in investable universe:
            get training data
            fit the model
            predict return for t+1
        
        #how much return we should make according to predictions
        top5 predicted returns at time t+1
        bottom5 predicted returns at time t+1
        predicted average return = top5 - bottom 5
        
        #how much return we actually make
        top5 actual returns at time t+1 =
        bottom5 actual returns at time t+1 = 
        actual average return = top5 - bottom5

In [9]:
#drop very first row
returns = returns.iloc[1:, :]

In [78]:
def get_investable(t, n_rows):
    "Find stocks in investable universe at time t\
    (stocks in the S&P500 that have prices recorded for the last n_rows days)"
    time = pd.to_datetime(t)
    
    df_investable = returns.copy(deep = True).sort_index(ascending = False)
    
    most_recent = time
    least_recent = time + pd.DateOffset(-n_rows)
    
    #take n_rows worth of data upto time specified
    df_investable = df_investable[most_recent:least_recent]
    
    #find all stocks that exist in the S&P at this time period
    investable_universe = []
    for col in df_investable.columns:
        if ~df_investable[col].iloc[:n_rows].isna().any():
            investable_universe.append(col)
        
    df_investable = df_investable[investable_universe]
    
    return df_investable

In [115]:
def get_train(df_investable, stock, t):
    stock = stock
    time = pd.to_datetime(t) + pd.DateOffset(-1)
        
    X = df_investable.loc[time:]

    #save the target (sample stock's returns upto time t)
    y = df_investable[[stock]]
    y.columns = [str(stock) + ' t+1']
    
    #concatenate outcome and features
    train = pd.concat([X, y], axis = 1).sort_index(ascending = False)
    
    #shit outcome down 1 row
    train[str(stock) + ' t+1'] = train[str(stock) + ' t+1'].shift(1)
    train = train.iloc[1:, :]

    #split into X_train and y_train
    X_train = train.drop(str(stock) + ' t+1', axis = 1)
    y_train = train[[str(stock) + ' t+1']]
    
    return X_train, y_train

In [4]:
def get_test(stock, t, n_rows):
    time = pd.to_datetime(t) + pd.DateOffset(1)
    
    test = get_investable(time, n_rows)
    
    #shift in the same way that training data is (with outcome as returns at t+1)
    X_test, y_test = get_training(test, stock, time)
    
    X_test = X_test.iloc[0:1,:]
    y_test = y_test.iloc[0:1,:]
    
    return X_test, y_test

In [194]:
def fit_model(X_train, y_train):
    model = LinearRegression()
    model.fit(X_train, y_train)
    return model

In [160]:
def pred_next(model, X_test, y_test):
    predictions = model.predict(X_test)
    df_eval = pd.concat([X_test, y_test], axis = 1)
    df_eval['predicted t+1'] = predictions
    return df_eval

In [2]:
def pred_next_all(t, investable):
    "Takes the investable universe at time t and returns predicted and actual returns for\
    all stocks in that universe"
    time = pd.to_datetime(t)

    #create a dataframe to hold the stock, actual return, predicted return at this time
    df = pd.DataFrame(columns = ['stock', 'actual return', 'pred return'])
    df['stock'] = investable.columns
    df.set_index('stock', inplace = True)

    for stock in investable.columns:
        #get train and test data
        X_train, y_train = get_train(investable, stock, time)
        X_test, y_test = get_test(stock, time, 80)
        
        #fit model
        model = fit_model(X_train, y_train)
        
        #predict
        predictions = pred_next(model, X_test, y_test)
        
        #add predictions and actual returns to dataframe
        df.loc[stock]['actual return'] = predictions.loc[time][str(stock) + ' t+1']
        df.loc[stock]['pred return'] = predictions.loc[time]['predicted t+1']

    return df

In [229]:
def pred_vs_act_return(df, n):
    "Takes dataframe with predicted and actual returns and outputs predicted portfolio returns\
    (based on strategy) and what the portfolio would have actually earned"
    pred = df.sort_values(by = 'pred return', ascending = False)
    topn_p = pred.head(n)
    botn_p = pred.tail(n)
    return_p = topn_p['pred return'].sum() - botn_p['pred return'].sum()
    return_a = topn_p['actual return'].sum() - botn_p['actual return'].sum()
    
    return return_p, return_a

In [209]:
def top_bottom_pred(df, n):
    "Takes dataframe with predicted and actual returns, and returns the topn and bottom n\
    predicted stocks"
    pred = df.sort_values(by = 'pred return', ascending = False)
    topn = list(pred.head(n).index)
    botn = list(pred.tail(n).index)
    return topn, botn

In [1]:
def pipeline(n_stocks):
    master_df = pd.DataFrame(
            columns = ['date', 'portfolio return'])
    
    time_list = ['2021-05-24','2021-05-25']
    master_df['date'] = time_list
    master_df.set_index('date', inplace = True)
    
    for time in time_list:
        #get the investable universe of stocks
        investable = get_investable(time, 80)

        #predict the next day's returns for each stock in investable
        df = pred_next_all(time, investable)

        #get predicted and actual returns based on predicted topn and bottomn
        pred_r, act_r = pred_vs_act_return(df, n_stocks)
        
        #format into master dataframe
        master_df.loc[time]['portfolio return'] = act_r
    
    return master_df

In [251]:
pipeline(5)

,predicted portfolio return,actual portfolio return
date,,
2021-05-24,42.2686,11.3629
2021-05-25,49.5955,-27.9481


In [218]:
time = pd.to_datetime('2020-02-24')
investable = get_investable(time, 80)
df = pred_next_all(time, investable)

UsageError: Line magic function `%%time` not found.


In [219]:
df

,actual return,pred return
stock,,
916328,-4.13565,-1.47219
936365,-3.15554,2.95392
905271,-4.08911,-1.1107
905113,-4.11272,-0.224207
905802,-4.69944,-0.237983
...,...,...
131745,-10.7679,-0.240092
69487D,-2.59594,-3.52746
68157P,-4.85015,-0.173878
